In [4]:
from gym_trading_env.downloader import download
import datetime
import pandas as pd
import glob
import numpy as np
import gymnasium as gym
import gym_trading_env
import torch

In [49]:
COIN_PAIRS=['BTC/USDT','ETH/USDT','SOL/USDT','BNB/USDT','XRP/USDT','ADA/USDT']

In [50]:
# download(exchange_names = ["binance"],
#     symbols= COIN_PAIRS,
#     timeframe= "1h",
#     dir = "data",
#     since= datetime.datetime(year= 2023, month= 1, day=1),
# )

In [37]:

# returns=[2,5,10,15,20,25,30]
files=glob.glob('data/**.pkl',)

# dates=[]
# for file in files:
#     data=pd.read_pickle(file)
#     for r in returns:
#         data[f'feature_log_return_{r}'] = np.log(1 + data.close.pct_change(r)+1e-15)
#         data[f'feature_log_volume_{r}'] = np.log(1 + data.volume.pct_change(r)+1e-15)
#         data = data.replace((np.inf, -np.inf,np.nan), 0)
#         data[f'feature_log_return_{r}']=data[f'feature_log_return_{r}'].values/np.linalg.norm(data[f'feature_log_return_{r}'].values)#.rolling(window=96).apply(compute_z)
#         data[f'feature_log_volume_{r}']=data[f'feature_log_volume_{r}'].values/np.linalg.norm(data[f'feature_log_volume_{r}'].values)#.rolling(window=96).apply(compute_z)
#     dates.append(data.reset_index()[['date_open']].rename(columns={'date_open':file.split('-')[1]}))
#     data = data.replace((np.inf, -np.inf), np.nan)
#     data=data.to_pickle(file)

In [53]:
def dynamic_feature_last_position_taken(history):
    return history['position', -1]

def dynamic_feature_real_position(history):
    return history['real_position', -1]

In [54]:
train_envs={}
test_envs={}
start_date=datetime.datetime(year= 2023, month= 9, day=1)
split_date=start_date+datetime.timedelta(days=7)
end_date=split_date+datetime.timedelta(days=7)

for file in files:
    data=pd.read_pickle(file)
    name=file.split('-')[1]
    train_data=data[start_date:split_date]
    train_envs[name] = gym.make("TradingEnv",
            name=name,
            df = train_data, # Your dataset with your custom features
            positions = [ -1, 0, 1], # -1 (=SHORT), 0(=OUT), +1 (=LONG)
            trading_fees = 0.01/100, # 0.01% per stock buy / sell (Binance fees)
            borrow_interest_rate= 0.0003/100, # 0.0003% per timestep (one timestep = 1h here)
            dynamic_feature_functions = [dynamic_feature_last_position_taken, dynamic_feature_real_position],
        )
    
    test_data=data[split_date:end_date]
    test_envs[name] = gym.make("TradingEnv",
            name=name,
            df = test_data, # Your dataset with your custom features
            positions = [ -.25, 0, .25], # -1 (=SHORT), 0(=OUT), +1 (=LONG)
            trading_fees = 0.01/100, # 0.01% per stock buy / sell (Binance fees)
            borrow_interest_rate= 0.0003/100, # 0.0003% per timestep (one timestep = 1h here)
            dynamic_feature_functions = [dynamic_feature_last_position_taken, dynamic_feature_real_position],
        )

In [55]:
env=train_envs['BTCUSDT']

In [64]:
obs=env.reset()
# env.action_space.sa
for i in range(5):
    act=env.action_space.sample()
    obs, rew, dun,trunc, info=env.step(act)
info

{'idx': 5,
 'step': 5,
 'date': numpy.datetime64('2023-09-01T05:00:00.000000000'),
 'position_index': 2,
 'position': 1,
 'real_position': 0.9999970006798082,
 'data_close': 26027.27,
 'data_low': 26025.4,
 'data_date_close': Timestamp('2023-09-01 06:00:00'),
 'data_high': 26057.54,
 'data_open': 26044.46,
 'data_volume': 557.51426,
 'portfolio_valuation': 1004.5350526213272,
 'portfolio_distribution_asset': 0.03859536707841662,
 'portfolio_distribution_fiat': 0.0030129222666346323,
 'portfolio_distribution_borrowed_asset': 0,
 'portfolio_distribution_borrowed_fiat': 0,
 'portfolio_distribution_interest_asset': 0.0,
 'portfolio_distribution_interest_fiat': 0.0,
 'reward': -0.0008571554522183696}

In [ ]:
class multiTradeEnv():
    def __init__(self,
                 pairs=[],
                 data_dict={},
                 positions=[-1,0,1],

                 ):
        self.envs={}
        self.pairs=pairs
        n_pairs=len(pairs)

        for name,data in data_dict.items():
            self.envs[name]=self.make_env(name,data,positions=positions)
        
        self.action_space=NotImplemented
        self.observation_space=NotImplemented


        def step(self,action):
            action=action/len(action)## normalize actions by the total so they are percentages of the total portfolio
            observations=[]
            rewards=[]
            dones=[]
            truncateds=[]
            infos={}
            for name,act in zip(action,self.pairs):
                obs, rew, dun,trunc, info=self.envs[name].step(act)
                observations.append(obs)
                rewards.append(rew)
                dones.append(dun)
                truncateds.append(trunc)
                for i in info.keys():
                    infos[f'{name}_{i}']=info[i]
            
            observation=np.array(observations).flatten()
            reward=sum(rewards)
            done=any(done)
            truncated=any(truncateds)
            return observation,reward,done,truncated,infos
        


        def reset(self):
            observations=[]
            for name,env in self.envs.items():
                obs=env.reset()
                observations.append(obs)

            observation=np.array(observations).flatten()
            return observation

        def make_env(self,name,data,positions=positions):
            env=gym.make("TradingEnv",
                name=name,
                df = data, # Your dataset with your custom features
                positions = [ -1, 0, 1], # -1 (=SHORT), 0(=OUT), +1 (=LONG)
                trading_fees = 0.01/100, # 0.01% per stock buy / sell (Binance fees)
                borrow_interest_rate= 0.0003/100, # 0.0003% per timestep (one timestep = 1h here)
                dynamic_feature_functions = [self.dynamic_feature_last_position_taken,self.dynamic_feature_real_position],
            )
            return env
    
    def dynamic_feature_last_position_taken(self,history):
        return history['position', -1]

    def dynamic_feature_real_position(self,history):
        return history['real_position', -1]